In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
import math
import tqdm

torch.manual_seed(192837)
rand = random.Random(192838)

In [2]:
class Xorer(nn.Module):
    def __init__(self,
                 n_digits: int=4,
                 hidden_dim: int=16,
                 hidden_layers: int=3):
        super().__init__()

        self.foot = nn.Linear(n_digits * 2, hidden_dim)
        self.body = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim) for _ in range(hidden_layers)])
        self.head = nn.Linear(hidden_dim, n_digits)
        self.relu = nn.ReLU(inplace=True)
        self.sigm = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.foot(x))
        for l in self.body:
            x = self.relu(l(x))
        x = self.sigm(self.head(x))
        return x

In [3]:
def to_bitvec(n: int, dim: int):
    return [(1 if ((n & (1 << b)) != 0) else 0) for b in range(dim)]

def from_bitvec(bits: list[int]):
    return sum([bits[b] * (1 << b) for b in range(len(bits))])

In [4]:
DIGITS = 4
TRAIN_TEST_SPLIT = 0.9

dataset_full = [(to_bitvec(i, DIGITS) + to_bitvec(j, DIGITS), to_bitvec(i ^ j, DIGITS)) for i in range(2 ** DIGITS) for j in range(2 ** DIGITS)]
rand.shuffle(dataset_full)
dataset_x, dataset_y = zip(*dataset_full)
dataset_x = torch.tensor(dataset_x, dtype=torch.float32)
dataset_y = torch.tensor(dataset_y, dtype=torch.float32)

n_full = len(dataset_full)
n_train = round(n_full * TRAIN_TEST_SPLIT)
n_test = n_full - n_train

train_x, test_x = dataset_x[:n_train, :], dataset_x[n_train:, :]
train_y, test_y = dataset_y[:n_train, :], dataset_y[n_train:, :]

print(n_full, n_train, n_test)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

256 230 26
torch.Size([230, 8]) torch.Size([26, 8]) torch.Size([230, 4]) torch.Size([26, 4])


In [5]:
loss_fn = F.binary_cross_entropy

def train_epoch(model: nn.Module,
                optimizer: torch.optim.Optimizer,
                dataset_x: torch.Tensor,
                dataset_y: torch.Tensor,
                batch_size: int,
                epoch_i: int,
                verbose: bool=False):
    model.train()

    n_batches = math.ceil(dataset_x.shape[0] / batch_size)
    loss_sum = 0.
    accu_sum = 0.

    for batch_i in tqdm.tqdm(range(n_batches), f'epoch {epoch_i} train') if verbose else range(n_batches):
        x = dataset_x[batch_i * batch_size:(batch_i + 1) * batch_size, :]
        y = dataset_y[batch_i * batch_size:(batch_i + 1) * batch_size, :]

        y_hat = model.forward(x)

        loss = loss_fn(y_hat, y)

        with torch.no_grad():
            accu = (torch.round(y_hat) == y).to(torch.float32).mean()
            accu_sum += accu.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_sum += loss.item()
    
    if verbose:
        print('train loss:', loss_sum / n_batches)
        print('train accu:', accu_sum / n_batches)

In [6]:
def test_epoch(model: nn.Module,
               dataset_x: torch.Tensor,
               dataset_y: torch.Tensor,
               batch_size: int,
               epoch_i: int,
               verbose: bool=False):
    model.train()

    n_batches = math.ceil(dataset_x.shape[0] / batch_size)
    loss_sum = 0.
    accu_sum = 0.

    with torch.no_grad():
        for batch_i in tqdm.tqdm(range(n_batches), f'epoch {epoch_i} test') if verbose else range(n_batches):
            x = dataset_x[batch_i * batch_size:(batch_i + 1) * batch_size, :]
            y = dataset_y[batch_i * batch_size:(batch_i + 1) * batch_size, :]

            y_hat = model.forward(x)

            loss = loss_fn(y_hat, y)

            accu = (torch.round(y_hat) == y).to(torch.float32).mean()
            accu_sum += accu.item()

            loss_sum += loss.item()

    if verbose:
        print(' test loss:', loss_sum / n_batches)
        print(' test accu:', accu_sum / n_batches)

In [7]:
model = Xorer(DIGITS, hidden_dim=16, hidden_layers=3)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
BATCH_SIZE = 8
N_EPOCHS = 400

In [8]:
for epoch_i in range(N_EPOCHS):
    train_epoch(model, optimizer, train_x, train_y, BATCH_SIZE, epoch_i, verbose=(epoch_i + 1) % 20 == 0)
    test_epoch(model, test_x, test_y, BATCH_SIZE, epoch_i, verbose=(epoch_i + 1) % 20 == 0)

epoch 19 train: 100%|██████████| 29/29 [00:00<00:00, 398.08it/s]


train loss: 0.5226443749049614
train accu: 0.6339798857425821


epoch 19 test: 100%|██████████| 4/4 [00:00<00:00, 1550.00it/s]


 test loss: 0.5279723852872849
 test accu: 0.5859375


epoch 39 train: 100%|██████████| 29/29 [00:00<00:00, 473.70it/s]


...

 test loss: 0.0026088613722095033
 test accu: 1.0


epoch 399 train: 100%|██████████| 29/29 [00:00<00:00, 537.35it/s]


train loss: 8.585623637090663e-05
train accu: 1.0


epoch 399 test: 100%|██████████| 4/4 [00:00<00:00, 2074.85it/s]

 test loss: 0.0020974984818167286
 test accu: 1.0


In [16]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

class AntiNet():
    def __init__(self,
               inner_net: nn.Module,
               input_shape: torch.Size,
               desired_output: torch.Tensor,
               input_min: float,
               input_max: float,
               lr: float=1e-2,
               temp_start: float=1.0,
               temp_end: float=0.05):
        super().__init__()

        self.input_shape = input_shape
        self.input = nn.Parameter(
            data=torch.normal(0, 1, size=input_shape, dtype=torch.float32),
            requires_grad=True
        )
        self.inner_net = inner_net
        self.grad_eraser = torch.optim.SGD(self.inner_net.parameters())
        self.anti_optimizer = torch.optim.Adam((self.input,), lr=lr)
        self.anti_scheduler = CosineAnnealingWarmRestarts(self.anti_optimizer, T_0=10, T_mult=2)

        self.desired_output = desired_output
        self.output_shape = self.desired_output.shape

        self.input_min = input_min
        self.input_max = input_max

        self.temp_start = temp_start
        self.temp_end = temp_end
        self.norm_func = lambda t: torch.softmax(t, dim=-1)

    def test(self):
        with torch.no_grad():
            return self.inner_net.forward(self.norm_func(self.input / self.temp_end))

    def zero_grad(self):
        self.grad_eraser.zero_grad()
        self.anti_optimizer.zero_grad()

    # def crop(self):
    #     self.input.data.copy_(torch.clamp(self.input.detach().clone(), self.input_min, self.input_max))

    def parabolic_loss(self, t: torch.Tensor):
        return (t * (1 - t)).sum()

    def step(self, step_i: int, max_steps: int):
        prev_input = self.input.detach().clone()

        inp_soft = self.norm_func(self.input / (self.temp_start + (self.temp_end - self.temp_start) / (max_steps - 1) * step_i))

        output = self.inner_net.forward(inp_soft)
        loss = F.binary_cross_entropy(output, self.desired_output) \
            + self.parabolic_loss(inp_soft)
        
        loss.backward()
        self.anti_optimizer.step()
        self.zero_grad()

        self.anti_scheduler.step()

        # self.crop()

        return prev_input, output.detach().clone(), loss.detach().clone()

In [17]:
ANTI_NET_STEPS = 3000
TEST_EXAMPLES = 5
TEST_TRIES = 5

for i in range(5):
    desired_input = test_x[i]
    desired_output = test_y[i]

    print(f'-=-=- step {i} -=-=-')
    print(f'desired output:', desired_output)
    print(f'example  input:', desired_input)

    fin_loss = float('+inf')
    fin_outp, fin_inp = None, None

    for try_i in range(TEST_TRIES):
        anti_net = AntiNet(model, (DIGITS * 2,), desired_output,
                       input_min=0.0, input_max=1.0,
                       lr=5e-2,
                       temp_start=1.0, temp_end=0.05)

        for step_i in tqdm.tqdm(range(ANTI_NET_STEPS), f'attempt #{try_i}'):
            inp, outp, loss = anti_net.step(step_i, ANTI_NET_STEPS)

        if loss.item() < fin_loss:
            fin_loss = loss.item()
            fin_outp = outp
            fin_inp = inp
        
        print(f'attempt #{try_i} loss:', loss.item())

    print(f'fin input:', fin_inp)
    print(f'fin input (sigm):', anti_net.norm_func(fin_inp / anti_net.temp_end))
    print(f'fin output:', fin_outp)
    print(f'fin loss:', fin_loss)

-=-=- step 0 -=-=-
desired output: tensor([0., 0., 1., 0.])
example  input: tensor([1., 1., 1., 0., 1., 1., 0., 0.])


attempt #0: 100%|██████████| 3000/3000 [00:08<00:00, 367.32it/s]


attempt #0 loss: 0.00014561894931830466


attempt #1: 100%|██████████| 3000/3000 [00:07<00:00, 395.11it/s]


attempt #1 loss: 27.003427505493164


...

attempt #2 loss: 4.267916679382324


attempt #3: 100%|██████████| 3000/3000 [00:07<00:00, 412.21it/s]


attempt #3 loss: 4.267916679382324


attempt #4: 100%|██████████| 3000/3000 [00:08<00:00, 354.04it/s]

attempt #4 loss: 4.267916679382324
fin input: tensor([-2.5934, -2.6062,  5.3710, -5.6458,  0.2409, -2.3132, -3.6352, -4.3053])
fin input (sigm): tensor([0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 2.8026e-45, 0.0000e+00,
        0.0000e+00, 0.0000e+00])
fin output: tensor([3.8542e-08, 1.1380e-04, 1.0000e+00, 3.7337e-05])
fin loss: 4.267916679382324
